In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord

In [3]:
from gammapy.data import DataStore
from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.makers import ReflectedRegionsBackgroundMaker
from gammapy.datasets import Datasets

In [4]:
from regions import CircleSkyRegion

In [5]:
path = '/Users/Leo/Documents/McGill/Summer Intro to VERITAS/BL Lacertae/data/'

In [6]:
datastore = DataStore.from_dir(path)
datastore.info()

Data store:
HDU index table:
BASE_DIR: /Users/Leo/Documents/McGill/Summer Intro to VERITAS/BL Lacertae/data
Rows: 65
OBS_ID: 83016 -- 83768
HDU_TYPE: ['aeff', 'edisp', 'events', 'gti', 'psf']
HDU_CLASS: ['aeff_2d', 'edisp_2d', 'events', 'gti', 'psf_table']


Observation table:
Observatory name: 'N/A'
Number of observations: 13



In [7]:
conf_1d = AnalysisConfig()

In [9]:
conf_1d.observations.obs_ids = [83016, 83017, 83018, 83019, 83020, 83021]

In [10]:
bllacertae_ra = (22 * u.deg + 2 * u.arcmin + 43.2913536816 * u.arcsec).to(u.deg)
bllacertae_dec = (42 * u.deg + 16 * u.arcmin + 39.979416792 * u.arcsec).to(u.deg)

In [11]:
conf_1d.datasets.stack = False

conf_1d.datasets.on_region = dict(
    frame="icrs",
    lon=bllacertae_ra,
    lat=bllacertae_dec,
    radius=0.1 * u.deg,
)

conf_1d.datasets.containment_correction = True

conf_1d.datasets.geom.axes.energy = dict(min=0.7 * u.TeV, max=10 * u.TeV, nbins=5)
conf_1d.datasets.geom.axes.energy_true = dict(min=0.3 * u.TeV, max=20 * u.TeV, nbins=20)

In [12]:
analysis_1d = Analysis(conf_1d)

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}


In [13]:
analysis_1d.config.observations.datastore = path

In [14]:
analysis_1d.get_observations()

Fetching observations.
Skipping run with missing HDUs; Required HDUs ['bkg'] not found in observation 83016
Skipping run with missing HDUs; Required HDUs ['bkg'] not found in observation 83017
Skipping run with missing HDUs; Required HDUs ['bkg'] not found in observation 83018
Skipping run with missing HDUs; Required HDUs ['bkg'] not found in observation 83019
Skipping run with missing HDUs; Required HDUs ['bkg'] not found in observation 83020
Skipping run with missing HDUs; Required HDUs ['bkg'] not found in observation 83021
Observations selected: 0 out of 6.
Number of selected observations: 0


In [23]:
exclusion_region = CircleSkyRegion(
    center=SkyCoord(bllacertae_ra, bllacertae_dec, frame="icrs"),
    radius=0.5 * u.deg,
)

target_position = SkyCoord(bllacertae_ra, bllacertae_dec, frame="icrs")

skydir = target_position.galactic
geom = WcsGeom.create(
    npix=(150, 150), binsz=0.05, skydir=skydir, proj="TAN", frame="icrs"
)

exclusion_mask = ~geom.region_mask([exclusion_region])

In [24]:
bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)

In [1]:
obs_ids = [83016, 83017, 83018, 83019, 83020, 83021]
observations = datastore.get_observations(obs_ids)

NameError: name 'datastore' is not defined

In [22]:
datasets = Datasets()

for obs_id, observation in zip(obs_ids, observations):
    dataset = dataset_maker.run(dataset_empty.copy(name=str(obs_id)), observation)
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    datasets.append(dataset_on_off)

In [16]:
from astropy.io import fits
from astropy.table import Table

hdul = fits.open(path + 'bl_combine.fits')

# Print out details for the fit
print (hdul[0].header)

SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                    8 / array data type                                NAXIS   =                    0 / number of array dimensions                     EXTEND  =                    T                                                  SOURCE  = 'BL Lac  '                                                            MODEL   = 'Power Law'                                                           NORM    = 1.15294722427777E-11 / cm^-2 s^-1 TeV^-1                              NORMERR = 9.57144680090104E-13 / cm^-2 s^-1 TeV^-1                              GAMMA   =   -3.414036385297607                                                  GAMMAERR=   0.1125493662640403                                                  ENORM   =                  1.0 / TeV                                            END                                                                                                                     

In [18]:
# Access the spectral points
spectrum = Table.read(hdul[1])
spectrum

E,dNdE,dNdE_errl,dNdE_erru,dNdE_ul,Hz,nuFnu,nuFnu_errl,nuFnu_erru,nuFnu_ul,Sigma
TeV,1 / (cm2 s TeV),1 / (cm2 s TeV),1 / (cm2 s TeV),1 / (cm2 s TeV),Hz,erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),erg / (cm2 s),
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.39810717055349704,2.594568396658546e-10,1.7531998788451783e-11,1.8328744267171375e-11,0.0,9.626188555952215e+25,6.588332629467718e-11,4.451863355250235e-12,4.654179248775704e-12,0.0,0.0
0.6309573444801928,6.225869491350621e-11,5.176365952855205e-12,5.47399791560092e-12,0.0,1.5256480711675739e+26,3.9710961850216795e-11,3.3016829402249626e-12,3.4915239180099986e-12,0.0,0.0
0.9999999999999992,1.2755113059947644e-11,1.6137508116126233e-12,1.7578911604462704e-12,0.0,2.417989242084916e+26,2.043594410867632e-11,2.5855138434642765e-12,2.8164521423821543e-12,0.0,0.0
1.584893192461112,1.7159468830440653e-12,4.112787859663585e-13,4.882137739821135e-13,0.0,3.83225468922459e+26,6.905803774837774e-12,1.655185612504906e-12,1.9648093752834976e-12,0.0,0.0
2.5118864315095766,0.0,0.0,0.0,5.61924080180172e-13,6.0737143687292305e+26,0.0,0.0,0.0,5.680519265456438e-12,0.0


In [19]:
# Access the confidence interval
model = Table.read(hdul[2])
model

E,dNdE,dNdE_err,Hz,nuFnu,nuFnu_err
TeV,1 / (cm2 s TeV),1 / (cm2 s TeV),Hz,erg / (cm2 s),erg / (cm2 s)
float64,float64,float64,float64,float64,float64
0.1,2.991202488009961e-08,5.899913095496186e-09,2.4179892420849182e+25,4.792434733852224e-10,9.4527029042346e-11
0.10023052380778996,2.9677805003696204e-08,5.846304457815454e-09,2.4235632829577234e+25,4.776856233296789e-10,9.410047639166174e-11
0.10046157902783952,2.9445419137217628e-08,5.7931742259271874e-09,2.4291501733217976e+25,4.761328372905075e-10,9.367570786664617e-11
0.10069316688518042,2.9214852919831455e-08,5.740518195123388e-09,2.4347499427982753e+25,4.745850988063927e-10,9.325271642847093e-11
0.10092528860766845,2.8986092103154698e-08,5.688332191612814e-09,2.4403626210765784e+25,4.730423914695283e-10,9.283149497412842e-11
0.10115794542598985,2.8759122550373417e-08,5.636612082037972e-09,2.445988237914567e+25,4.715046989254446e-10,9.241203649540168e-11
...,...,...,...,...,...
19.952623149687348,4.2030264899862294e-16,1.7004974991278624e-16,4.8245228127318496e+27,2.6808500375914826e-13,1.0846419348825204e-13
